In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

script that contains collection of approximate 2RDMs.

The following function caculates the electron electron interactio  energy from the Fock basis representation of the 2RDM $\gamma_2$ in the follwong way:
$$E_{ee} = \sum_p \sum_q \sum_r \sum_s \gamma_{2_{pqrs}} \langle p q | r s \rangle$$

In [2]:
def ee_from_gamma_2(eri,dm2):
    E = 0 
    dim = eri.shape[0]
    for p in range(0,dim):
        for q in range(0,dim):
            for r in range(0,dim):
                for s in range(0,dim):
                    E+=eri[p,q,r,s]*dm2[p,q,r,s]
    return E

In [3]:
def HF_2RDM(n, M):
    # this form is for spin restriced orbitals 
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==k and j==l:
                        TWORDM[i,k,j,l] = n[i]*n[j]
                    if i==l and j==k:
                        TWORDM[i,k,j,l] -= TWORDM[i,k,j,l]

    return TWORDM

In [4]:
def MU_2RDM(n, M):
    # this form is for spin restriced orbitals 
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==k and j==l:
                        TWORDM[i,k,j,l] = n[i]*n[j]
                    if i==l and j==k:
                        TWORDM[i,k,j,l] -= np.sqrt(n[i]*n[j])
    return TWORDM

In [5]:
def POW_2RDM(n, M, alpha):
    # this form is for spin restriced orbitals 
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==k and j==l:
                        TWORDM[i,k,j,l] = n[i]*n[j]
                    if i==l and j==k:
                        TWORDM[i,k,j,l] -= (n[i]*n[j])**(alpha)
    return TWORDM

In [6]:
def GU_2RDM(n, M):
    # this form is for spin restriced orbitals 
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==k and j==l and j != k :
                        TWORDM[i,k,j,l] +=  n[i]*n[j]
                    if i==l and j==k and i != j :
                        TWORDM[i,k,j,l] -= .5*np.sqrt(n[i]*n[j])
                    if i==j and j==k and k==l :
                        TWORDM[i,k,j,l] += (n[i]-.5)*.5*n[i]


    return TWORDM

In [7]:
mol = gto.Mole()
mol.atom = """
    O    0.    0.    0.
"""
#mol.basis = "cc-pvdz"
mol.basis = "6-31g" 
mol.build()

In [8]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()
h = mf.get_hcore()
C = mf.mo_coeff
h1 = C.T@h@C

converged SCF energy = -74.6558129536031


In [9]:
# Find electron-repulsion integrals (eri).
eri = ao2mo.kernel(mol, mf.mo_coeff)
h2_MO = np.asarray(ao2mo.restore(1, eri, mol.nao))

In [10]:
# First create FCI solver with function fci.FCI and solve the FCI problem
cisolver = fci.FCI(mol, mf.mo_coeff)
e, fcivec = cisolver.kernel()

In [11]:
dm1, dm2 = cisolver.make_rdm12(fcivec, mf.mo_coeff.shape[1], mol.nelec)
FCIoccu, FCInaturalC = np.linalg.eigh(dm1)

In [12]:
print(.5*ee_from_gamma_2(h2_MO,dm2), e-np.trace(np.matmul(dm1,h1)))

28.23071205706589 28.230712057065915


In [13]:
eri_fciNAO = h2_MO.copy()
for i in range(4):
  eri_fciNAO = np.tensordot(eri_fciNAO, FCInaturalC, axes=1).transpose(3, 0, 1, 2)


In [14]:
hf_dm2 = HF_2RDM(FCIoccu, h.shape[0])
mu_dm2 = MU_2RDM(FCIoccu, h.shape[0])
gu_dm2 = GU_2RDM(FCIoccu, h.shape[0])

In [15]:
print(.5*ee_from_gamma_2(eri_fciNAO,hf_dm2), e-np.trace(np.matmul(dm1,h1)))
print(.5*ee_from_gamma_2(eri_fciNAO,mu_dm2), e-np.trace(np.matmul(dm1,h1)))
print(.5*ee_from_gamma_2(eri_fciNAO,gu_dm2), e-np.trace(np.matmul(dm1,h1)))

23.059968712706297 28.230712057065915
28.015393442712423 28.230712057065915
27.329547873832635 28.230712057065915
